<a href="https://colab.research.google.com/github/Kalima83/procesamiento_lenguaje_natural_Desafios/blob/main/desafio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de lenguaje con tokenización por caracteres
## Consigna

1. Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.

2. Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.

3. Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.

4. Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.

## Sugerencias
Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.

Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU. rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.

In [1]:
import random
import io
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy

## Datos
Utilizaremos como dataset un libro en castellano "El Conde de Montecristo" de
Alejandro Dumas.

In [2]:
# descargar de textos.info
import urllib.request

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

In [3]:
raw_html = urllib.request.urlopen('https://www.textos.info/alejandro-dumas/el-conde-de-montecristo/ebook')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag )
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text + ' '

# pasar todo el texto a minúscula
article_text = article_text.lower()

In [4]:
# en article text se encuentra el texto de todo el libro
article_text[:1000]

' el 24 de febrero de 1815, el vigía de nuestra señora de la guarda dio\n la señal de que se hallaba a la vista el bergantín el faraón procedente\n de esmirna, trieste y nápoles. como suele hacerse en tales casos, salió\n inmediatamente en su busca un práctico, que pasó por delante del \ncastillo de if y subió a bordo del buque entre la isla de rión y el cabo\n mongión. en un instante, y también como de costum\xadbre, se llenó de \ncuriosos la plataforma del castillo de san juan, por\xadque en marsella se \ndaba gran importancia a la llegada de un buque y sobre todo si le \nsucedía lo que al faraón, cuyo casco había salido de los astilleros de \nla antigua focia y pertenecía a un naviero de la ciudad. mientras tanto, el buque seguía avanzando; habiendo pasado \nfeliz\xadmente el estrecho producido por alguna erupción volcánica entre \nlas islas de calasapeigne y de jaros, dobló la punta de pomegue \nhendien\xaddo las olas bajo sus tres gavias, su gran foque y la mesana. lo \nhacía con 

In [5]:
# seleccionamos el tamaño de contexto
max_context_size = 100


# Usaremos las utilidades de procesamiento de textos y secuencias de Keras
from tensorflow.keras.utils import pad_sequences # se utilizará para padding


# en este caso el vocabulario es el conjunto único de caracteres que existe en todo el texto
chars_vocab = set(article_text)


# la longitud de vocabulario de caracteres es:
print("La longitud de vocabulario de caracteres es: ")
len(chars_vocab)


La longitud de vocabulario de caracteres es: 


83

In [6]:
# Construimos los dicionarios que asignan índices a caracteres y viceversa.
# El diccionario `char2idx` servirá como tokenizador.
char2idx = {k: v for v,k in enumerate(chars_vocab)}
idx2char = {v: k for k,v in char2idx.items()}

# Tokenizar

In [7]:
# tokenizamos el texto completo
# convierte todo el texto del libro en una secuencia de números,
# usando el diccionario char2idx
tokenized_text = [char2idx[ch] for ch in article_text]

print("Primeros 1000 caracteres del libro convertidos en índices numéricos")
print(tokenized_text[:1000])

Primeros 1000 caracteres del libro convertidos en índices numéricos
[68, 54, 44, 68, 51, 15, 68, 12, 54, 68, 3, 54, 38, 64, 54, 64, 78, 68, 12, 54, 68, 9, 60, 9, 66, 18, 68, 54, 44, 68, 79, 33, 46, 16, 80, 68, 12, 54, 68, 28, 63, 54, 75, 10, 64, 80, 68, 75, 54, 45, 78, 64, 80, 68, 12, 54, 68, 44, 80, 68, 46, 63, 80, 64, 12, 80, 68, 12, 33, 78, 71, 68, 44, 80, 68, 75, 54, 45, 80, 44, 68, 12, 54, 68, 61, 63, 54, 68, 75, 54, 68, 77, 80, 44, 44, 80, 38, 80, 68, 80, 68, 44, 80, 68, 79, 33, 75, 10, 80, 68, 54, 44, 68, 38, 54, 64, 46, 80, 28, 10, 16, 28, 68, 54, 44, 68, 3, 80, 64, 80, 47, 28, 68, 74, 64, 78, 29, 54, 12, 54, 28, 10, 54, 71, 68, 12, 54, 68, 54, 75, 48, 33, 64, 28, 80, 18, 68, 10, 64, 33, 54, 75, 10, 54, 68, 17, 68, 28, 32, 74, 78, 44, 54, 75, 40, 68, 29, 78, 48, 78, 68, 75, 63, 54, 44, 54, 68, 77, 80, 29, 54, 64, 75, 54, 68, 54, 28, 68, 10, 80, 44, 54, 75, 68, 29, 80, 75, 78, 75, 18, 68, 75, 80, 44, 33, 47, 71, 68, 33, 28, 48, 54, 12, 33, 80, 10, 80, 48, 54, 28, 10, 54, 68, 54,

# Organizando y estructurando el dataset

In [8]:
# separaremos el dataset entre entrenamiento y validación.
# `p_val` será la proporción del corpus que se reservará para validación
# `num_val` es la cantidad de secuencias de tamaño `max_context_size` que se usará en validación
p_val = 0.1
num_val = int(np.ceil(len(tokenized_text)*p_val/max_context_size))

In [9]:
# separamos la porción de texto utilizada en entrenamiento de la de validación.
train_text = tokenized_text[:-num_val*max_context_size]
val_text = tokenized_text[-num_val*max_context_size:]

In [10]:
# Train con ventana deslizante (sliding window)
tokenized_sentences_val = [val_text[init*max_context_size:init*(max_context_size+1)] for init in range(num_val)]

In [11]:
# Val con segmentación
tokenized_sentences_train = [train_text[init:init+max_context_size]
                             for init in range(len(train_text)-max_context_size+1)]


In [12]:
# Generamos los arrays
X = np.array(tokenized_sentences_train[:-1])
y = np.array(tokenized_sentences_train[1:])

In [13]:
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,68,54,44,68,51,15,68,12,54,68,...,68,77,80,44,44,80,38,80,68,80
1,54,44,68,51,15,68,12,54,68,3,...,77,80,44,44,80,38,80,68,80,68
2,44,68,51,15,68,12,54,68,3,54,...,80,44,44,80,38,80,68,80,68,44
3,68,51,15,68,12,54,68,3,54,38,...,44,44,80,38,80,68,80,68,44,80
4,51,15,68,12,54,68,3,54,38,64,...,44,80,38,80,68,80,68,44,80,68


In [14]:
pd.DataFrame(y).head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,54,44,68,51,15,68,12,54,68,3,...,77,80,44,44,80,38,80,68,80,68
1,44,68,51,15,68,12,54,68,3,54,...,80,44,44,80,38,80,68,80,68,44
2,68,51,15,68,12,54,68,3,54,38,...,44,44,80,38,80,68,80,68,44,80
3,51,15,68,12,54,68,3,54,38,64,...,44,80,38,80,68,80,68,44,80,68
4,15,68,12,54,68,3,54,38,64,54,...,80,38,80,68,80,68,44,80,68,79


In [15]:
vocab_size = len(chars_vocab)

In [16]:
print("Shapes:", X.shape, y.shape)

Shapes: (2294651, 100) (2294651, 100)


# Definiendo el modelo
Utilizo los modelos recomendados

In [17]:
from tensorflow.keras import backend as K
import gc
K.clear_session()
gc.collect()

0

In [18]:
class PplCallback(keras.callbacks.Callback):

    '''
    Este callback es una solución ad-hoc para calcular al final de cada epoch de
    entrenamiento la métrica de Perplejidad sobre un conjunto de datos de validación.
    La perplejidad es una métrica cuantitativa para evaluar la calidad de la generación de secuencias.
    Además implementa la finalización del entrenamiento (Early Stopping)
    si la perplejidad no mejora después de `patience` epochs.
    '''

    def __init__(self, val_data, history_ppl,patience=5):
      # El callback lo inicializamos con secuencias de validación sobre las cuales
      # mediremos la perplejidad
      self.val_data = val_data

      self.target = []
      self.padded = []

      count = 0
      self.info = []
      self.min_score = np.inf
      self.patience_counter = 0
      self.patience = patience

      # nos movemos en todas las secuencias de los datos de validación
      for seq in self.val_data:

        len_seq = len(seq)
        # armamos todas las subsecuencias
        subseq = [seq[:i] for i in range(1,len_seq)]
        self.target.extend([seq[i] for i in range(1,len_seq)])

        if len(subseq)!=0:

          self.padded.append(pad_sequences(subseq, maxlen=max_context_size, padding='pre'))

          self.info.append((count,count+len_seq))
          count += len_seq

      self.padded = np.vstack(self.padded)


    def on_epoch_end(self, epoch, logs=None):

        # en `scores` iremos guardando la perplejidad de cada secuencia
        scores = []

        predictions = self.model.predict(self.padded,verbose=0)

        # para cada secuencia de validación
        for start,end in self.info:

          # en `probs` iremos guardando las probabilidades de los términos target
          probs = [predictions[idx_seq,-1,idx_vocab] for idx_seq, idx_vocab in zip(range(start,end),self.target[start:end])]

          # calculamos la perplejidad por medio de logaritmos
          scores.append(np.exp(-np.sum(np.log(probs))/(end-start)))

        # promediamos todos los scores e imprimimos el valor promedio
        current_score = np.mean(scores)
        history_ppl.append(current_score)
        print(f'\n mean perplexity: {current_score} \n')

        # chequeamos si tenemos que detener el entrenamiento
        if current_score < self.min_score:
          self.min_score = current_score
          self.model.save("my_model.keras")
          print("Saved new model!")
          self.patience_counter = 0
        else:
          self.patience_counter += 1
          if self.patience_counter == self.patience:
            print("Stopping training...")
            self.model.stop_training = True


## SimpleRNN

In [23]:

from keras.layers import Input, TimeDistributed, CategoryEncoding, SimpleRNN, Dense
from keras.models import Model, Sequential

model = Sequential()

model.add(TimeDistributed(CategoryEncoding(num_tokens=vocab_size, output_mode = "one_hot"),input_shape=(None,1)))
model.add(SimpleRNN(200, return_sequences=True, dropout=0.1, recurrent_dropout=0.1 ))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, None, 83)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, None, 200)      │        56,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 83)       │        16,683 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,483 (287.04 KB)

 Trainable params: 73,483 (287.04 KB)

 Non-trainable params: 0 (0.00 B)

## Entrenar usando el callback

In [ ]:
# fiteamos, nótese el agregado del callback con su inicialización. El batch_size lo podemos seleccionar a mano
# en general, lo mejor es escoger el batch más grande posible que minimice el tiempo de cada época.
# En la variable `history_ppl` se guardarán los valores de perplejidad para cada época.
history_ppl = []
hist = model.fit(X, y, epochs=20, callbacks=[PplCallback(tokenized_sentences_val,history_ppl)], batch_size=256)

In [1]:
# Entrenamiento
epoch_count = range(1, len(history_ppl) + 1)
sns.lineplot(x=epoch_count,  y=history_ppl)
plt.show()

NameError: name 'model' is not defined

In [ ]:
# Cargamos el mejor modelo guardado del entrenamiento para hacer inferencia
model = keras.models.load_model('my_model.keras')

# Funciones de generación

## Greedy Search

In [ ]:
def generate_greedy(model, seed, length=300):
    seq = [char2idx[c] for c in seed]
    for _ in range(length):
        x = np.array(seq[-max_context_size:])[None, :]
        preds = model.predict(x, verbose=0)[0][-1]
        next_char = np.argmax(preds)
        seq.append(next_char)
    return "".join(idx2char[i] for i in seq)

## Sampling con temperatura

In [ ]:
def sample_with_temperature(preds, T=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-9) / T
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_with_temperature(model, seed, T=1.0, length=300):
    seq = [char2idx[c] for c in seed]
    for _ in range(length):
        x = np.array(seq[-max_context_size:])[None, :]
        preds = model.predict(x, verbose=0)[0][-1]
        next_char = sample_with_temperature(preds, T)
        seq.append(next_char)
    return "".join(idx2char[i] for i in seq)

## Beam Search determinístico

In [ ]:
def beam_search(model, seed, k=3, length=200):
    seq = [char2idx[c] for c in seed]
    beams = [(seq, 0)]

    for _ in range(length):
        new_beams = []
        for seq, score in beams:
            x = np.array(seq[-max_context_size:])[None, :]
            preds = model.predict(x, verbose=0)[0][-1]
            top_k = np.argsort(preds)[-k:]

            for token in top_k:
                new_seq = seq + [token]
                new_score = score + np.log(preds[token] + 1e-9)
                new_beams.append((new_seq, new_score))

        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:k]

    best_seq = beams[0][0]
    return "".join(idx2char[i] for i in best_seq)

In [ ]:
#Comprobación
print("===== GREEDY =====")
print(generate_greedy(model, seed, 300))

print("\n===== BEAM SEARCH (k=3) =====")
print(beam_search(model, seed, beam_width=3, length=300))

print("\n===== TEMPERATURE 0.7 =====")
print(sample_with_temperature(model, seed, length=300, T=0.7))

print("\n===== TEMPERATURE 1.2 =====")
print(sample_with_temperature(model, seed, length=300, T=1.2))


## LSTM

In [27]:
from tensorflow.keras.layers import LSTM

model = Sequential([
    Embedding(vocab_size, embed_dim, input_length=max_context_size),
    LSTM(rnn_units, return_sequences=False),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(0.001)
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


## GRU

In [28]:
from tensorflow.keras.layers import GRU

model = Sequential([
    Embedding(vocab_size, embed_dim, input_length=max_context_size),
    GRU(rnn_units, return_sequences=False),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(0.001)
)